In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：批处理预测自定义训练图像分类模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/master/notebooks/official/automl/sdk_custom_image_classification_batch.ipynb">
      在 Google Cloud 笔记本中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Vertex SDK来训练和部署用于批量预测的自定义图像分类模型。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已内置于TensorFlow中。训练后的模型会预测一幅图像属于十种类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将使用Vertex SDK在Google预构建的Docker容器中，从Python脚本创建一个带有训练管道的自定义模型，然后对上传的模型进行批量预测。您还可以选择使用`gcloud`命令行工具或在Cloud Console上在线创建自定义模型。

执行的步骤包括：

- 为训练模型创建Vertex自定义作业。
- 训练TensorFlow模型。
- 检索和加载模型工件。
- 查看模型评估。
- 将模型上传为Vertex `Model`资源。
- 进行批量预测。

成本

本教程使用Google Cloud的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage定价](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估计。

### 设置您的本地开发环境

如果您正在使用Colab或Google Cloud笔记本，则您的环境已满足运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合此笔记本的要求。您需要以下内容：

- 云存储SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3的虚拟环境中运行的Jupyter笔记本

云存储指南[设置Python开发环境](https://cloud.google.com/python/setup) 和 [Jupyter安装指南](https://jupyter.org/install) 提供了满足这些要求的详细说明。以下步骤提供了一套简化的说明：

1. [安装并初始化SDK](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) 并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开本笔记本。

## 安装

安装最新版本的Python的Vertex SDK。

In [ ]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install --upgrade tensorflow $USER_FLAG

### 重新启动内核

一旦您安装了额外的软件包，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始前

### GPU运行时

本教程不需要GPU运行时。

### 设置您的Google Cloud项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得$300的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI API、Compute Engine API和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter运行以`!`开头的行作为shell命令，并插入以`$`开头的Python变量。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于在笔记本的其余部分中进行操作。以下是Vertex AI支持的区域。我们建议您选择离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能不会使用多区域存储桶来训练Vertex AI。并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在一个现场教程会话中，您可能正在使用一个共享的测试帐户或项目。为了避免用户在创建的资源上发生名称冲突，您为每个实例会话创建一个时间戳，并将该时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的 Google Cloud 帐户

**如果您正在使用 Google Cloud Notebook**，您的环境已经通过验证。跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按提示进行身份验证以通过 oAuth 认证您的帐户。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**单击创建服务帐号**。

在**服务帐号名称**字段中，输入一个名称，并单击**创建**。

在**授予此服务帐号对项目的访问权限**部分，单击角色下拉列表。在筛选框中键入“Vertex”，然后选择**Vertex 管理员**。在筛选框中键入“Storage Object Admin”，然后选择**Storage Object Admin**。

单击创建。包含您的密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中将您的服务帐号密钥的路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建云存储桶

**无论您使用的笔记本环境是什么，都需要执行以下步骤。**

当您初始化用于Python的Vertex SDK时，您需要指定一个云存储暂存桶。该暂存桶是您的数据集和模型资源在不同会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在本教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

初始化用于Python的Vertex SDK

为您的项目和相应的存储桶初始化Python的Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_NAME)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 来使用支持GPU的容器映像，并指定分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

    （aip.AcceleratorType.NVIDIA_TESLA_K80，4）

否则，指定 `(None, None)` 来使用一个在CPU上运行的容器映像。

在[这里](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)了解有关您地区硬件加速器支持的更多信息

*注意*：在TF 2.3之前发布的GPU支持版本将无法在此教程中加载自定义模型。这是一个已知问题，已在TF 2.3中修复--由服务函数中生成的静态图操作引起。如果您在自己的自定义模型中遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置您将用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB的内存。
     - `n1-highmem`: 每个vCPU 6.5GB的内存
     - `n1-highcpu`: 每个vCPU 0.9GB的内存
 - `vCPUs`: \[2, 4, 8, 16, 32, 64, 96 \]的数量

*注意：以下不支持用于训练：*

 - `standard`: 2个vCPUs
 - `highcpu`: 2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在您已经准备好开始为CIFAR10创建自己的定制模型和训练。

### 检查培训包

#### 包布局

在开始培训之前，您将查看一个如何组装用于自定义培训作业的Python包。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义培训作业的Python脚本。*注意*，当我们在工作进程池规范中提到它时，我们会将目录斜杠替换为点（`trainer.task`），并删除文件后缀（`.py`）。

#### 包装配

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py内容

在下一个单元格中，您将编写训练脚本task.py的内容。我们不会详细讨论，这只是让您浏览一下。总结一下：

- 从命令行(`--model_dir`)获取要保存模型工件的目录，如果未指定，则从环境变量`AIP_MODEL_DIR`中获取。
- 从TF Datasets（tfds）加载CIFAR10数据集。
- 使用TF.Keras模型API构建模型。
- 编译模型（`compile()`）。
- 根据参数`args.distribute`设置训练分布策略。
- 根据参数`args.epochs`和`args.steps`训练模型（`fit()`）。
- 将训练好的模型(`save(args.model_dir)`)保存到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，将培训文件夹打包成压缩的tar文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 创建并运行自定义训练任务

要训练自定义模型，您需要执行两个步骤：1）创建自定义训练任务，2）运行任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建自定义训练任务，并设置以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如，pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aip.CustomTrainingJob(
    display_name="cifar10_" + TIMESTAMP,
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备你的命令行参数

现在为你的自定义训练容器定义命令行参数：

- `args`: 传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`: 对于我们的演示，我们使用这个命令行参数来指定存储模型工件的位置。
      - 直接: 将云存储位置作为命令行参数传递给你的训练脚本（设置变量 `DIRECT = True`），或
      - 间接: 服务将云存储位置作为环境变量 `AIP_MODEL_DIR` 传递给你的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，你需要在作业规范中告诉服务模型工件的位置。
  - `"--epochs=" + EPOCHS`: 训练的轮数。
  - `"--steps=" + STEPS`: 每轮的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_NAME, TIMESTAMP)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

#### 运行自定义训练作业

接下来，您可以通过调用 `run` 方法并传入以下参数来运行自定义作业以启动训练作业：

- `args`：要传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点的加速器数量。
- `base_output_dir`：用于写入模型工件的 Google Cloud 存储位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载保存的模型

您的模型以TensorFlow SavedModel格式存储在Cloud Storage存储桶中。现在从Cloud Storage存储桶中加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载模型，您可以使用TF.Keras的`model.load_model()`方法，并传递保存模型的Cloud Storage路径——由`MODEL_DIR`指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将从 `tf.keras.datasets` 加载 CIFAR10 测试（留存）数据，使用 `load_data()` 方法。这将返回数据集的元组，有两个元素。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和对应的标签。

您不需要训练数据，因此我们用 `(_, _)` 加载它。

在您可以运行数据进行评估之前，您需要预处理它：

`x_test`:
1. 通过将每个像素除以 255 来归一化（重新缩放）像素数据。这将用介于 0 到 1 之间的 32 位浮点数字替换每个单字节整数像素。

`y_test`:<br/>
2. 标签目前是标量（稀疏）的。如果您回顾一下 `trainer/task.py` 脚本中的 `compile()` 步骤，您会发现它已经为稀疏标签编译。所以我们不需要再做其他的操作。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估一下定制作业中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

### 图像数据的服务功能

要将图像传递给预测服务，您需要将压缩（例如 JPEG）图像字节编码为 base 64 -- 这样在通过网络传输二进制数据时，可以确保内容不受修改。由于该部署模型期望输入数据为原始（未压缩）字节，您需要确保将 base 64 编码的数据在传递给部署模型之前转换回原始字节。

为了解决这个问题，定义一个服务功能（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个 `@tf.function` 装饰器，以便服务功能与底层模型融合（而不是在 CPU 上游处理）。

当您发送预测或解释请求时，请求的内容会被 base 64 解码为一个 Tensorflow 字符串（`tf.string`），然后传递给服务功能（`serving_fn`）。该服务功能将`tf.string`预处理为原始（未压缩）的 numpy 字节（`preprocess_fn`），以匹配模型的输入要求：
- `io.decode_jpeg`- 解压缩 JPG 图像，返回一个具有三个通道（RGB）的 Tensorflow 张量。
- `image.convert_image_dtype` - 将整数像素值更改为 float 32 类型。
- `image.resize` - 调整图像大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据缩放（标准化）到 0 到 1 之间。

此时，数据可以传递给模型（`m_call`）。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model, model_path_to_deploy, signatures={"serving_default": serving_fn}
)

## 获取serving函数的签名

您可以通过重新加载模型到内存中，并查询每个层对应的签名来获取模型的输入和输出层的签名。

对于您的目的，您需要获取serving函数的签名。为什么呢？当我们将数据作为HTTP请求包发送进行预测时，图像数据是Base64编码的，而我们的TF.Keras模型需要numpy输入。您的serving函数将会从Base64转换为numpy数组。

当进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--这将在您进行预测请求时使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到一个`Model`资源，需要使用以下参数：

- `display_name`: 用于`Model`资源的可读名称。
- `artifact`: 已训练模型构件的云存储位置。
- `serving_container_image_uri`: 服务容器镜像。
- `sync`: 是否异步或同步执行上传。

如果使用异步方式运行`upload()`方法，随后可以使用`wait()`方法阻塞等待完成。

In [ ]:
model = aip.Model.upload(
    display_name="cifar10_" + TIMESTAMP,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=False,
)

model.wait()

获取测试项目

您将使用数据集的测试（留出）部分中的示例作为测试项目。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

准备请求内容
您将以压缩的JPG图像形式发送CIFAR10图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘为压缩的JPEG图像。
- 将图像数据从\[0,1)范围反标准化为[0,255)。
- 将32位浮点值转换为8位无符号整数。

In [ ]:
import cv2

cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_NAME/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_NAME/tmp2.jpg

test_item_1 = BUCKET_NAME + "/tmp1.jpg"
test_item_2 = BUCKET_NAME + "/tmp2.jpg"

### 制作批量输入文件

现在制作一个批处理输入文件，你将把它存储在您本地的云存储桶中。批处理输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）制作一行一个字典条目。该字典包含键/值对：

- `input_name`：底层模型的输入层的名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：压缩后的JPG图像字节，作为base64编码的字符串。

预测请求中的每个实例都是一个形式为的字典条目：

                        {serving_input: {'b64': content}}

为了将图像数据传递给预测服务，您将字节编码为base64 - 这样在通过网络传输二进制数据时可以确保内容不被修改。

- `tf.io.read_file`：将压缩后的JPG图像作为原始字节读入内存。
- `base64.b64encode`：将原始字节编码为base64编码的字符串。

In [ ]:
import base64
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 进行批量预测请求

现在您的模型资源已经训练完成，您可以通过调用 batch_predict() 方法来进行批量预测，使用以下参数：

- `job_display_name`：批量预测工作的可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作复制品的加速器数量。
- `sync`：如果设置为True，调用将阻塞，直到异步批处理作业完成。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="cifar10_" + TIMESTAMP,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_NAME,
    instances_format="jsonl",
    predictions_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的Cloud Storage输出存储桶。您可以调用iter_outputs()方法获取包含结果的每个Cloud Storage文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `instance`: 预测请求。
- `prediction`: 预测响应。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源:

- 数据集
- 流水线
- 模型
- 端点
- AutoML训练作业
- 批量作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_all = True

if delete_all:
    # Delete the dataset using the Vertex dataset object
    try:
        if "dataset" in globals():
            dataset.delete()
    except Exception as e:
        print(e)

    # Delete the model using the Vertex model object
    try:
        if "model" in globals():
            model.delete()
    except Exception as e:
        print(e)

    # Delete the endpoint using the Vertex endpoint object
    try:
        if "endpoint" in globals():
            endpoint.delete()
    except Exception as e:
        print(e)

    # Delete the AutoML or Pipeline trainig job
    try:
        if "dag" in globals():
            dag.delete()
    except Exception as e:
        print(e)

    # Delete the custom trainig job
    try:
        if "job" in globals():
            job.delete()
    except Exception as e:
        print(e)

    # Delete the batch prediction job using the Vertex batch prediction object
    try:
        if "batch_predict_job" in globals():
            batch_predict_job.delete()
    except Exception as e:
        print(e)

    # Delete the hyperparameter tuning job using the Vertex hyperparameter tuning object
    try:
        if "hpt_job" in globals():
            hpt_job.delete()
    except Exception as e:
        print(e)

    if "BUCKET_NAME" in globals():
        ! gsutil rm -r $BUCKET_NAME